# Inputs loading

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%cd /Users/fedor.grigoryev/mycode

/Users/fedor.grigoryev/mycode


In [3]:
tcga_directory = '~/projs/tcga/data/cohorts'

expanded_directory = os.path.expanduser(tcga_directory)
cohorts = os.listdir(expanded_directory)
cohorts = [file for file in cohorts if not file.startswith('.')]

In [8]:
coh = cohorts[0]

In [9]:
ls ~/projs/tcga/data/cohorts/$coh

clin_annotation.tsv  file_annotation.tsv  tpm.tsv
counts.tsv           qc.tsv


In [10]:
coh = cohorts[0]

tpm = pd.read_csv(f'{expanded_directory}/{coh}/tpm.tsv', sep='\t', index_col=0)
qc = pd.read_csv(f'{expanded_directory}/{coh}/qc.tsv', sep='\t', index_col=0)
ann = pd.read_csv(f'{expanded_directory}/{coh}/clin_annotation.tsv', sep='\t', index_col=0)
file_anot = pd.read_csv(f'{expanded_directory}/{coh}/file_annotation.tsv', sep='\t', index_col=0)

In [11]:
tpm.shape

(19938, 553)

In [12]:
ann.shape

(1002, 158)

In [7]:
ann.head()

,case_id,case_submitter_id,project_id,age_at_index,age_is_obfuscated,cause_of_death,cause_of_death_source,country_of_residence_at_enrollment,days_to_birth,days_to_death,...,treatment_arm,treatment_dose,treatment_dose_units,treatment_effect,treatment_effect_indicator,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
112,005669e5-1a31-45fb-ae97-9d450e74e7cb,TCGA-77-A5GF,TCGA-LUSC,70,'--,'--,'--,'--,-25654,840,...,'--,'--,'--,'--,'--,'--,'--,no,'--,"Pharmaceutical Therapy, NOS"
113,005669e5-1a31-45fb-ae97-9d450e74e7cb,TCGA-77-A5GF,TCGA-LUSC,70,'--,'--,'--,'--,-25654,840,...,'--,'--,'--,'--,'--,'--,'--,no,'--,"Radiation Therapy, NOS"
288,00fd9306-4a68-49ab-a768-e5fed126a765,TCGA-NC-A5HJ,TCGA-LUSC,59,'--,'--,'--,'--,-21730,418,...,'--,'--,'--,'--,'--,'--,'--,no,'--,"Radiation Therapy, NOS"
289,00fd9306-4a68-49ab-a768-e5fed126a765,TCGA-NC-A5HJ,TCGA-LUSC,59,'--,'--,'--,'--,-21730,418,...,'--,'--,'--,'--,'--,'--,'--,no,'--,"Pharmaceutical Therapy, NOS"
357,01417822-b608-4934-8fe0-594315212be5,TCGA-85-7843,TCGA-LUSC,50,'--,'--,'--,'--,-18353,'--,...,'--,'--,'--,'--,'--,'--,'--,not reported,'--,"Pharmaceutical Therapy, NOS"


In [118]:
new_file_ids = list(qc.index)

In [119]:
%cd tcga_api_calls

[Errno 2] No such file or directory: 'tcga_api_calls'
/Users/fedor.grigoryev/mycode/tcga_api_calls


In [11]:
payload_backbone_json = {
    "filters":{
        "op":"in",
        "content":{
            "field":"files.file_id",
            "value":[
                "0001801b-54b0-4551-8d7a-d66fb59429bf",
                "002c67f2-ff52-4246-9d65-a3f69df6789e"
            ]
        }
    },
    "format":"TSV",
    "fields":"file_id,file_name,cases.submitter_id,cases.case_id,data_category,data_type,cases.samples.tumor_descriptor,cases.samples.tissue_type,cases.samples.sample_type,cases.samples.submitter_id,cases.samples.sample_id,cases.samples.portions.analytes.aliquots.aliquot_id,cases.samples.portions.analytes.aliquots.submitter_id",
    "size":"10000"
}

payload_backbone_json['filters']['content']['value'] 

['0001801b-54b0-4551-8d7a-d66fb59429bf',
 '002c67f2-ff52-4246-9d65-a3f69df6789e']

In [12]:
import json

def update_payload_file_ids(json_backbone, ids):
    json_backbone['filters']['content']['value'] = ids
    
    with open(json_file, 'w') as f:
        json.dump(json_backbone, f, indent=4)
    return json_backbone

json_file = 'pl_files.txt'
payload_json = update_payload_file_ids(payload_backbone_json, new_file_ids)

In [125]:
# the file query example:
! curl --request POST --header "Content-Type: application/json" --data @pl_files.txt 'https://api.gdc.cancer.gov/files' > response_files.tsv
# also the cases query example:
# ! curl --request POST --header "Content-Type: application/json" --data @pl_samples.txt 'https://api.gdc.cancer.gov/files' > response_samples.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  236k  100  207k  100 29333   158k  22427  0:00:01  0:00:01 --:--:--  180k


In [107]:
def rename_fnc(col):
    if 'cases' in col:
        if 'submitter_id' in col:
            return '.'.join(col.replace('.0', '').split('.')[-2:])
        else:
            return col.split('.')[-1]
    else:
        return col

resp_df = pd.read_csv('response_files.tsv', sep='\t', index_col='id')

cols = list(map(rename_fnc, resp_df.columns))
resp_df.columns = cols

resp_df.to_csv(f'{expanded_directory}/{coh}/file_annotation.tsv', sep='\t')

In [109]:
set(resp_df.index) ^ set(new_file_ids)

set()

In [122]:
from tqdm.notebook import tqdm

unmatched_id_cohs = []

for coh in tqdm(cohorts):
    if coh != 'TARGET-NBL':
        print(f'Running for {coh}:')
        qc = pd.read_csv(f'{expanded_directory}/{coh}/qc.tsv', sep='\t', index_col=0)
        
        new_file_ids = list(qc.index)
        print(f'quering {len(new_file_ids)} indices:')
        payload_json = update_payload_file_ids(payload_backbone_json, new_file_ids)
        ! curl --request POST --header "Content-Type: application/json" --data @pl_files.txt 'https://api.gdc.cancer.gov/files' > response_files.tsv
    
        resp_df = pd.read_csv('response_files.tsv', sep='\t', index_col='id')
        cols = list(map(rename_fnc, resp_df.columns))
        resp_df.columns = cols
        print(f'received {resp_df.shape} values')
        
        if len(set(resp_df.index) ^ set(new_file_ids)) == 0:
            print('ids to response matched completely!\n')
        else:
            unmatched_id_cohs.append(coh)
            print('unmatched ids!\n')
        
        resp_df.to_csv(f'{expanded_directory}/{coh}/file_annotation.tsv', sep='\t')

  0%|          | 0/65 [00:00<?, ?it/s]

Running for TCGA-LUSC:
quering 553 indices:
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  249k  100  219k  100 30928   154k  21827  0:00:01  0:00:01 --:--:--  176k
received (553, 13) values
ids to response matched completely!

Running for TCGA-STAD:
quering 448 indices:
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  202k  100  177k  100 25153   142k  20198  0:00:01  0:00:01 --:--:--  162k
received (448, 13) values
ids to response matched completely!

Running for TCGA-UVM:
quering 80 indices:
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37596  100 32683  100  4913  46522   6993 --:--:-- --:--:-- --:--:-- 53479
received (80, 13) values
ids to r

[]

In [70]:
! cat response_samples.tsv | head

analysis.workflow_type	cases.0.case_id	cases.0.project.project_id	cases.0.samples.0.portions.0.analytes.0.aliquots.0.aliquot_id	cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id	cases.0.samples.0.sample_id	cases.0.samples.0.sample_type	cases.0.samples.0.submitter_id	cases.0.samples.0.tissue_type	cases.0.samples.0.tumor_descriptor	cases.0.submitter_id	data_category	data_type	file_id	file_name	id
STAR - Counts	001e0309-9c50-42b0-9e38-347883ee2cd3	TCGA-UCEC	ab241aac-4962-4ccd-919c-140031a13dba	TCGA-D1-A17N-01A-11R-A12I-07	f93931ba-8333-4e22-a430-ca02d9da3ffc	Primary Tumor	TCGA-D1-A17N-01A	Tumor	Primary	TCGA-D1-A17N	Transcriptome Profiling	Gene Expression Quantification	7476fbef-f6b9-439c-9589-a123a0ed9db2	2109e5d5-43e6-4ca8-bd8c-8f34489437e8.rna_seq.augmented_star_gene_counts.tsv	7476fbef-f6b9-439c-9589-a123a0ed9db2
STAR - Counts	0be4fa90-0122-4b26-b35f-7b1a4a16e63b	TCGA-THYM	d6f2ac55-00ea-43b0-8879-278c78eb339b	TCGA-4V-A9QM-01A-11R-A42C-07	f70b46da-2ca0-471c-8322-1c2cda0b16